In [1]:
from sqlalchemy import text
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Database credentials
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
db_name = 'project3_db'

In [3]:
# Establishing a connection to the database
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db_name}')

In [4]:
query = text("SELECT * FROM cleandata_table")
with engine.connect() as conn:
    print(conn.execute(query))

In [5]:
with engine.connect() as connection:
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df.head())

   id area_code  quarter  year  establishments  total_employed total_wages  \
0   1      1001        1  2020            1778           66384   238572054   
1   2      1003        1  2020           13072          452026  1439657260   
2   3      1005        1  2020            1128           49570   160195240   
3   4      1007        1  2020             730           29156   110388552   
4   5      1009        1  2020            1568           51336   152730444   

  taxable_wage  average_weekly_wage  %chg_lq_estabishments  %chg_total_emp  \
0     97010816                 4319                   9.20        1.136000   
1    749596298                 4103                   4.58        0.780000   
2     80797450                 4011                  12.67        1.164000   
3     48400892                 4670                  16.92        1.237333   
4     72113324                 3616                   9.58        1.042667   

   %chge_lq_total_qtrly_wages  %chge_lq_taxable_wages  %chge_a

In [6]:
# Formatting numeric columns for better readability
df['total_employed'] = df['total_employed'].apply(lambda x: "{:,}".format(x))
df['total_wages'] = df['total_wages'].apply(lambda x: "{:,}".format(x))
df['taxable_wage'] = df['taxable_wage'].apply(lambda x: "{:,}".format(x))

In [7]:
# Rename columns for clarity
df.rename(columns={
    '%chg_lq_estabishments': 'pct_change_lq_establishments',
    '%chg_total_emp': 'pct_change_total_emp',
    '%chge_lq_total_qtrly_wages': 'pct_change_lq_total_qtrly_wages',
    '%chge_lq_taxable_wages': 'pct_change_lq_taxable_wages',
    '%chge_avg_wkly_wage': 'pct_change_avg_weekly_wage'
}, inplace=True)

In [8]:
# Convert percentages from decimal to percentage format
for col in ['pct_change_lq_establishments', 'pct_change_total_emp', 
            'pct_change_lq_total_qtrly_wages', 'pct_change_lq_taxable_wages', 
            'pct_change_avg_weekly_wage']:
    df[col] = df[col].apply(lambda x: "{:.2%}".format(x))

In [9]:
# Display the cleaned DataFrame
print(df.head())

   id area_code  quarter  year  establishments total_employed    total_wages  \
0   1      1001        1  2020            1778         66,384    238,572,054   
1   2      1003        1  2020           13072        452,026  1,439,657,260   
2   3      1005        1  2020            1128         49,570    160,195,240   
3   4      1007        1  2020             730         29,156    110,388,552   
4   5      1009        1  2020            1568         51,336    152,730,444   

  taxable_wage  average_weekly_wage pct_change_lq_establishments  \
0   97,010,816                 4319                      920.00%   
1  749,596,298                 4103                      458.00%   
2   80,797,450                 4011                     1267.00%   
3   48,400,892                 4670                     1692.00%   
4   72,113,324                 3616                      958.00%   

  pct_change_total_emp pct_change_lq_total_qtrly_wages  \
0              113.60%                         119.0

In [10]:
# Export the DataFrame to a CSV file
df.to_csv('exported_dataframe.csv', index=False, encoding='utf-8')